In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, n_informative=10, n_redundant=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=y, train_size=0.8)

X_train_t = torch.tensor(X_train, dtype=torch.float)
X_test_t = torch.tensor(X_test, dtype=torch.float)
y_train_t = torch.tensor(y_train, dtype=torch.float)
y_test_t = torch.tensor(y_test, dtype=torch.float)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=True)


In [13]:
class NetClassifier(nn.Module):
    def __init__(self):
        super(NetClassifier, self).__init__()
        self.linear = nn.Linear(10, 32)
        self.linear2 = nn.Linear(32, 48)
        self.linear3 = nn.Linear(48, 1)
    
    def forward(self, x):
        x = F.relu(self.linear(x))
        x = F.relu(self.linear2(x))
        x = torch.sigmoid(self.linear3(x))
        return x
    
model = NetClassifier().to(device)
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=1e-3)
    

In [28]:
epochs = 5

for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_accuracy = 0
    for samples, targets in train_loader:
        samples, targets = samples.to(device), targets.to(device)
        
        optimizer.zero_grad()
        output = model.forward(samples)
        loss = criterion(output, targets)
        outputs = [1 if item > 0.5 else 0 for item in output.squeeze().tolist()]
        train_accuracy += sum(np.array(outputs)==np.array(targets.tolist()))
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    else:
        train_loss /= len(train_ds)
        train_accuracy /= len(train_ds)
    
    model.eval()
    with torch.no_grad():
        test_loss = 0
        test_accuracy = 0
        for samples, targets in test_loader:
            samples, targets = samples.to(device), targets.to(device)
            output = model.forward(samples)
            outputs = [1 if item > 0.5 else 0 for item in output.squeeze().tolist()]
            test_accuracy += sum(np.array(outputs)==np.array(targets.tolist()))
            test_loss += loss.item()
        else:
            test_loss /= len(test_ds)
            test_accuracy /= len(test_ds)
    
    print(f"Epoch: {epoch} | Train loss: {train_loss:.6f} | Train accuracy: {train_accuracy:.4f} | "
          f"Test loss: {test_loss:.6f} | Test accuracy: {test_accuracy:.4f}")  


Epoch: 0 | Train loss: 0.143062 | Train accuracy: 0.9525 | Test loss: 0.094058 | Test accuracy: 0.9450
Epoch: 1 | Train loss: 0.140087 | Train accuracy: 0.9487 | Test loss: 0.050826 | Test accuracy: 0.9400
Epoch: 2 | Train loss: 0.136174 | Train accuracy: 0.9550 | Test loss: 0.269123 | Test accuracy: 0.9450
Epoch: 3 | Train loss: 0.132743 | Train accuracy: 0.9600 | Test loss: 0.086309 | Test accuracy: 0.9350
Epoch: 4 | Train loss: 0.129618 | Train accuracy: 0.9575 | Test loss: 0.097866 | Test accuracy: 0.9350


In [31]:
result = model.forward(X_train_t[0:1].to(device))
result.item()

0.8214522004127502